# Pre-processing pipeline

In [ ]:
import mne
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import concurrent.futures
import pywt
import scipy as sp
from sklearn.model_selection import StratifiedGroupKFold

### configuration setup

In [ ]:
RAW_PATH = '/Users/jannis/Git/tuh-eeg-seizure-detection/data/raw'
OUTPUT_PATH = '/Users/jannis/Git/tuh-eeg-seizure-detection/data/processed_fast'

SAMPLING_FREQ = 250
WINDOW_LENGTH = 10
CONFIGURATIONS = ["01_tcp_ar"]
CHANNELS = ["EEG FP1-REF", "EEG FP2-REF", "EEG F7-REF", "EEG F3-REF", "EEG F4-REF", "EEG F8-REF", "EEG T3-REF", "EEG C3-REF", "EEG C4-REF", "EEG T4-REF", "EEG T5-REF", "EEG P3-REF", "EEG P4-REF", "EEG T6-REF", "EEG O1-REF", "EEG O2-REF", "EEG CZ-REF", "EEG A1-REF", "EEG A2-REF"]

### load raw data

#### extract events from annotations

In [ ]:
def extract_events_from_annotations(annotation_file):
    with open(annotation_file, "r") as f:
        annotations = f.readlines()
        events = annotations[6:] 
        
        data = []
        for event in events:
            parts = event.split(",")
            
            start_time = float(parts[1])
            stop_time = float(parts[2])
            label = parts[3]
            
            data.append({
                "label": label,
                "onset": start_time,
                "duration": stop_time - start_time
            })
            
            
    return data

#### load the TUH EEG dataset

In [ ]:
def load_tuh_eeg():
    cols = ["set", "patient_id", "session_id", "configuration", "recording_id", "recording_path", "label", "onset", "duration"]
    data = []
    
    # get all edf files in RAW/edf
    edf_path = os.path.join(RAW_PATH, "edf")
    for root, dirs, files in os.walk(edf_path):
        for file in files:
            if file.endswith(".edf"):
                rel_path = os.path.relpath(root, edf_path)
                parts = rel_path.split("/")
                
                if len(parts) != 4:
                    continue
                    
                set_name, patient_id, session_id, configuration = parts
                
                if configuration not in CONFIGURATIONS:
                    continue
                
                recording_path = os.path.join(root, file)
                recording_id = file.replace(".edf", "").split("_")[-1]
                annotation_path = recording_path.replace(".edf", ".csv_bi")
                
                if not os.path.exists(recording_path) or not os.path.exists(annotation_path):
                    continue
                
                events = extract_events_from_annotations(annotation_path)
                for event in events:
                    data.append({
                        "set": set_name,
                        "patient_id": patient_id,
                        "session_id": session_id,
                        "configuration": configuration,
                        "recording_id": recording_id,
                        "recording_path": recording_path,
                        "label": event["label"],
                        "onset": event["onset"],
                        "duration": event["duration"]
                    })
                    
    return pd.DataFrame(data, columns=cols)

data = load_tuh_eeg()
data.tail()

### train/test split

In [ ]:
windows = []

for i, event in data.iterrows():
    patient_id = event["patient_id"]
    onset = event["onset"]
    duration = event["duration"]
    label = event["label"]

    num_windows = int(duration / WINDOW_LENGTH)

    if num_windows == 0:
        continue
        
    for i in range(num_windows):
        windows.append({
            "patient_id": patient_id,
            "onset": onset + i * WINDOW_LENGTH,
            "duration": WINDOW_LENGTH,
            "label": label
        })

windows = pd.DataFrame(windows)
x = np.array(windows["duration"])
y = np.array(windows["label"])
groups = np.array(windows["patient_id"])

cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

splits = list(cv.split(x, y, groups))
test_fold_idx = np.random.choice(len(splits))

train_idx, test_idx = splits[test_fold_idx]

x_train, y_train = x[train_idx], y[train_idx]
x_test, y_test = x[test_idx], y[test_idx]

# Get the number of positive/negative samples in both train and test and their ratio
unique, counts = np.unique(y_train, return_counts=True)
train_ratio = counts[1] / counts[0]
print(f"Train - Unique: {unique} Counts: {counts}")
print(f"Train ratio: {train_ratio}")

unique, counts = np.unique(y_test, return_counts=True)
test_ratio = counts[1] / counts[0]
print(f"Test - Unique: {unique} Counts: {counts}")
print(f"Test ratio: {test_ratio}")


### feature extraction

In [ ]:
def calc_coeffs_features(coeffs):
    mean = np.mean(coeffs)
    median = np.median(coeffs)
    std = np.std(coeffs)
    skew = sp.stats.skew(coeffs)
    kurtosis = sp.stats.kurtosis(coeffs)
    rms = np.sqrt(np.mean(coeffs**2))
    
    return {
        "mean": mean,
        "median": median,
        "std": std,
        "skew": skew,
        "kurtosis": kurtosis,
        "rms": rms
    } 
    
def extract_wavelet_features(channel_data):
    cA5, cD5, cD4, cD3, cD2, cD1 = pywt.wavedec(channel_data, 'db4', level=5)
    
    d3_features = calc_coeffs_features(cD3)
    d4_features = calc_coeffs_features(cD4)
    d5_features = calc_coeffs_features(cD5)
    a5_features = calc_coeffs_features(cA5)
    
    # rename keys to include the level
    d3_features = {f"d3_{k}": v for k, v in d3_features.items()}
    d4_features = {f"d4_{k}": v for k, v in d4_features.items()}
    d5_features = {f"d5_{k}": v for k, v in d5_features.items()}
    a5_features = {f"a5_{k}": v for k, v in a5_features.items()}
    
    features = {}
    features.update(a5_features)
    features.update(d3_features)
    features.update(d4_features)
    features.update(d5_features) 
     
    return features

def extract_power_features(channel_data, sfreq=SAMPLING_FREQ, n_fft=256):
    frequency_bands = {
        "delta": (0.5, 4),
        "theta": (4, 7),
        "alpha": (7, 12),
        "beta": (12, 30),
        "gamma": (30, 50)
    }
    
    power_features = {}
    band_powers = {}
    
    psds, freqs = mne.time_frequency.psd_array_welch(channel_data, sfreq=sfreq, n_fft=n_fft, fmin=0.5, fmax=50)

    # Calculate power within each frequency band
    for band, (fmin, fmax) in frequency_bands.items():
        # Find indices of frequencies within the band
        band_indices = np.where((freqs >= fmin) & (freqs <= fmax))[0]
        # Sum the power spectral density values within the band
        band_power = np.sum(psds[band_indices])
        power_features[f'power_{band}'] = band_power
        band_powers[band] = band_power  # Store the band power for ratio calculations

    # Calculate power ratios
    alpha_beta_ratio = band_powers['alpha'] / band_powers['beta'] if band_powers['beta'] != 0 else np.nan
    theta_beta_ratio = band_powers['theta'] / band_powers['beta'] if band_powers['beta'] != 0 else np.nan
    theta_alpha_beta_ratio = (band_powers['theta'] + band_powers['alpha']) / band_powers['beta'] if band_powers['beta'] != 0 else np.nan
    theta_alpha_beta_alpha_ratio = (band_powers['theta'] + band_powers['alpha']) / (band_powers['beta'] + band_powers['alpha']) if (band_powers['beta'] + band_powers['alpha']) != 0 else np.nan
    alpha_theta_ratio = band_powers['alpha'] / band_powers['theta'] if band_powers['theta'] != 0 else np.nan
    theta_alpha_ratio = band_powers['theta'] / band_powers['alpha'] if band_powers['alpha'] != 0 else np.nan

    power_features['alpha_beta_ratio'] = alpha_beta_ratio
    power_features['theta_beta_ratio'] = theta_beta_ratio
    power_features['(theta+alpha)_beta_ratio'] = theta_alpha_beta_ratio
    power_features['(theta+alpha)/(beta+alpha)_ratio'] = theta_alpha_beta_alpha_ratio
    power_features['alpha_theta_ratio'] = alpha_theta_ratio
    power_features['theta_alpha_ratio'] = theta_alpha_ratio
    
    return power_features


### pre-process the data

In [ ]:
def preprocess(patient_id):
    corrupted = []
    windows = []
    
    # output directory
    seizure_output_dir = os.path.join(OUTPUT_PATH, patient_id, "seizure")
    non_seizure_output_dir = os.path.join(OUTPUT_PATH, patient_id, "non_seizure")
    os.makedirs(seizure_output_dir, exist_ok=True)
    os.makedirs(non_seizure_output_dir, exist_ok=True)
    
    recordings = data[data["patient_id"] == patient_id]["recording_path"].unique()
    
    for recording in recordings:
        raw = mne.io.read_raw_edf(recording, preload=True).pick_channels(CHANNELS)
        
        # sometimes meas date breaks code
        raw.set_meas_date(None)
        
        # pre-processing to remove noise
        raw.resample(SAMPLING_FREQ)
    
        events = data[data["recording_path"] == recording]
        
        onset = events["onset"].values
        duration = events["duration"].values
        label = events["label"].values
        
        annotations = mne.Annotations(onset=onset, duration=duration, description=label)
        raw.set_annotations(annotations)
        
        for i, event in events.iterrows():
            patient_id = event["patient_id"]
            onset = event["onset"]
            duration = event["duration"]
            label = event["label"]
            
            num_windows = int(duration / WINDOW_LENGTH)
            
            if num_windows == 0:
                continue
                
            if onset + duration > raw.times[-1]:
                if onset + duration - 1/SAMPLING_FREQ == raw.times[-1]:
                    raw_event = raw.copy().crop(onset, raw.times[-1], include_tmax=True)
                else:
                    print("Corrupted annotation", patient_id, event["session_id"], event["recording_id"])
                    corrupted.append((patient_id, event["session_id"], event["recording_id"]))
                    continue
            else:
                raw_event = raw.copy().crop(onset, onset + duration, include_tmax=False)
                
            epochs = mne.make_fixed_length_epochs(raw_event, duration=WINDOW_LENGTH)
            
            for epoch in epochs:
                epoch_features = {}
                for i, channel in enumerate(raw.info["ch_names"]):
                    wavelet_features = extract_wavelet_features(epoch[i])
                    power_features = extract_power_features(epoch[i])
                    
                    for key, value in wavelet_features.items():
                        epoch_features[f"{channel}_{key}"] = value
                    for key, value in power_features.items():
                        epoch_features[f"{channel}_{key}"] = value
                
                epoch_features["label"] = label
                epoch_features["patient_id"] = patient_id
                windows.append(epoch_features)
               
            #output_dir = seizure_output_dir if label == "seiz" else non_seizure_output_dir
            #file_name = f"{patient_id}_{event['session_id']}_{event["recording_id"]}_{i}_epo.fif"
            #epochs.save(os.path.join(output_dir, file_name), overwrite=True)
            raw_event.close()
            
        raw.close()
        
    return pd.DataFrame(windows), corrupted

In [ ]:
patients = data["patient_id"].unique()
patients = patients[200:210]
extracted_features = []
corrupted = []
for patient in patients:
    features, corrupted = preprocess(patient)
    extracted_features.append(features)
    corrupted.extend(corrupted)
    
extracted_features = pd.concat(extracted_features)
extracted_features

In [ ]:
wavelet_features.shape